In [6]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
from keras.models import load_model, Model
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2

import random

def pass_arg(nsim, tr_size, dropoutrate):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)


    # import pickle

    # def save_obj(obj, name):
    #     with open(name, 'wb') as f:
    #         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn


    def phy_loss_mean(params):
        # useful for cross-checking training
        loss1, loss2, loss3, loss4, lam1, lam2 = params
        x1, x2, x3 = loss1*(loss1>0), loss2*(loss2>0), loss3*(loss3>0)
    #     print(np.mean(x1), x1.shape[0])
    #     print(np.mean(x2), x2.shape[0])
    #     print(np.mean(x3), x3.shape[0])

        if x1.any() and x1.shape[0]>1:
            X_scaled1 = (x1 - np.min(x1)) / (np.max(x1) - np.min(x1))
            x1 = X_scaled1
        if x2.any() and x2.shape[0]>1:
            X_scaled2 = (x2 - np.min(x2)) / (np.max(x2) - np.min(x2))
            x2 = X_scaled2
        if x3.any() and x3.shape[0]>1:
            X_scaled3 = (x3 - np.min(x3)) / (np.max(x3) - np.min(x3))
            x3 = X_scaled3
        return (lam1*np.mean(x1) + lam2*np.mean(x2) + lam2*np.mean(x3))
    #     return (lam1*np.mean(x1) + lam2*np.mean(x2) + lam2*np.mean(x3) + lam2*loss4)

    def PGNN_train_test(optimizer_name, optimizer_val, drop_rate, iteration, n_layers, n_nodes, tr_size, pre_train):

        # Hyper-parameters of the training process
    #     batch_size = int(tr_size/2)
        batch_size = 10
        num_epochs = 300
        val_frac = 0.25
        patience_val = 80

        # Initializing results filename
        exp_name = "Pre-train" + optimizer_name + '_drop' + str(drop_rate) + '_nL' + str(n_layers) + '_nN' + str(n_nodes) + '_trsize' + str(tr_size) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../results/'
        model_name = results_dir + exp_name + '.h5' # storing the trained model
        results_name = results_dir + exp_name + '_results.dat' # storing the results of the model
        
        # Load labeled data
        # data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
        
        # data1 = data[:1303, :]
        # data2 = data[-6:, :]
        # datah = np.vstack((data1,data2))
        # np.random.shuffle(datah)
        # x_unlabeled = datah[:, :2] # 1303 last regular sample
        # y_unlabeled = datah[:, -3:-1]

        # Load labeled data
        data = np.loadtxt('../data/labeled_data.dat')
        x_labeled = data[:, :2] # -2 because we do not need porosity predictions
        y_labeled = data[:, -2:-1] # dimensionless bond length and

        # normalize dataset with MinMaxScaler
        scaler = preprocessing.MinMaxScaler(feature_range=(0, 1.0))
    #     scaler = preprocessing.StandardScaler()
        x_labeled = scaler.fit_transform(x_labeled)
        # x_unlabeled = scaler.fit_transform(x_unlabeled)
        # y_labeled = scaler.fit_transform(y_labeled)

        # train and test data
        trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
#         testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]
        testX, testY = x_labeled[30:,:], y_labeled[30:]

        dependencies = {'root_mean_squared_error': root_mean_squared_error}

        # load the pre-trained model using non-calibrated physics-based model predictions (./data/unlabeled.dat)
        loaded_model = load_model(results_dir + pre_train, custom_objects=dependencies)

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.001, l2=.001)))
            # model.add(Dropout(rate=drop_rate))
            model.add(MCDropout(rate=drop_rate))
        model.add(Dense(1, activation='linear'))

        # pass the weights to all layers but 1st input layer, whose dimensions are updated
        for new_layer, layer in zip(model.layers[1:], loaded_model.layers[1:]):
            new_layer.set_weights(layer.get_weights())

        model.compile(loss='mean_squared_error',
                      optimizer=optimizer_val,
                      metrics=[root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val,verbose=1)

        print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=0,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])

        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

            
        test_scores = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
#             predictions = model.predict(testX)
#             samples.append(predictions)
            test_score = model.evaluate(testX, testY, verbose=1)
            test_scores.append(test_score[1])
        return np.array(test_scores)
    


    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 1
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_rate = dropoutrate # Fraction of nodes to be dropped out
        n_layers = 2 # Number of hidden layers
        n_nodes = 5 # Number of nodes per hidden layer

        # # Iterating over different training fractions and splitting indices for train-test splits
        # trsize_range = [4,6,8,10,20]

        # #default training size = 5000
        # tr_size = trsize_range[4]
        
        # pre-trained model
        pre_train = 'Poro_Pre-trainAdadelta_drop0_nL2_nN5_trsize1308_iter0.h5'
        tr_size = int(tr_size)

#         # use regularizer
#         reg = True

#         #set lamda=0 for pgnn0
#         lamda = [1, 1] # Physics-based regularization constant

        # total number of runs
        iter_range = np.arange(1)
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
            # results, result_file, pred, obs, rmse = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                            # iteration, n_layers, n_nodes, tr_size, lamda, reg)
            # testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                        iteration, n_layers, n_nodes, tr_size, pre_train)
    
    return np.squeeze(pred)


In [7]:
mean_rmses=[]
std_rmses=[]
# for ii in ([.005,.01,.02,.05,.1,.15,.2,.25,.3,0.5]):
for ii in ([5,10,15,20,30]):
    test_rmse = pass_arg(50, ii, 0.05)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)

Tr_size: 5
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.004705499857664108, 0.017763651907444]
9/9 [==============================] - 0s 111us/step
Tr_size: 10
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.003831044305115938, 0.011233353056013584]
9/9 [==============================] - 0s 111us/step
Tr_size: 15
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.0004809132660739124, 0.015418878756463528]
9/9 [==============================] - 0s 0us/step


9/9 [==============================] - 0s 111us/step
Tr_size: 20
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.000388483633287251, 0.013521510176360607]
9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0002809964935295284, 0.013321500271558762]
9/9 [==============================] - 0s 111us/step


In [8]:
mean_rmses

[0.013819292970001697,
 0.008959805900231004,
 0.014267750959843397,
 0.013454186040908098,
 0.013600420970469713]

In [9]:
std_rmses

[0.002429259320554208,
 0.001850709734460213,
 0.0007153421020521668,
 0.0005223524645082339,
 0.0009140530362462718]

In [5]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mean_rmses, "../mean_rmse_dnn_upd_MC.dat")
save_obj(std_rmses, "../std_rmse_dnn_upd_MC.dat")

In [3]:
mc_pred=np.mean(pred,axis=0)
mc_pred

array([ 2.62061860e-02,  1.87012106e-02,  2.03988180e-02,  1.67454164e-02,
        1.15877548e-02,  1.77968144e-02, -1.27896736e-03, -1.31999478e-02,
        2.60994807e-02,  1.33960219e-02,  2.77881604e-02,  9.34297452e-04,
        6.04256568e-03,  1.26129938e-02,  7.02638784e-03,  2.18978971e-02,
        2.32310016e-02, -4.19802684e-03,  1.63644198e-02,  2.18827017e-02,
        2.80083790e-02,  2.09308118e-02,  2.16732565e-02,  1.84855182e-02,
        1.77870262e-02,  3.32493819e-02,  1.94747355e-02, -9.24617343e-04,
       -1.61748938e-02,  4.68686111e-02,  1.53396726e-02,  3.43751386e-02,
        1.31887319e-02,  2.40315683e-02,  2.57186349e-02,  1.02430042e-02,
        4.46724612e-03,  7.93455448e-03,  3.40867825e-02,  1.84419602e-02,
        9.89243481e-03,  2.23036781e-02,  2.42859274e-02,  2.76430584e-02,
        2.40111295e-02,  1.80043541e-02,  1.87582728e-02,  2.57925484e-02,
        1.78488772e-02,  2.23268848e-02, -3.57775800e-02,  1.67228747e-02,
        2.13348549e-02,  

In [4]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_upd_MC_Xx.dat")